In [1]:
'''
A template to see how template_car_2 is performing
No training is required since the car is completely hardcoded
'''

from template_2_car import MyCar
import numpy as np

track1 = 'tracks/sample_track_1.jpg'
track2 = 'tracks/sample_track_2.jpg'
track3 = 'tracks/sample_track_3.jpg'

car = MyCar(id=0, weights=np.zeros(6,)) # weights are dummy as we are not using them in our move function at all

# Visualize on different tracks
f1 = car.run(track1, save='template_2_track1.gif')
f2 = car.run(track2, save='template_2_track2.gif')
f3 = car.run(track3, save='template_2_track3.gif')
print(f'Overall fitness: {f1+f2+f3} = {f1} + {f2} + {f3}')

# save (dummy) weights
car.save(file='template_2_weights')


/home/aadit/anaconda3/lib/python3.7/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
Visualizing fitness for car 0: 100%|██████████| 100/100 [00:03<00:00, 29.49it/s]


Overall fitness: 2.4620000000000015 = 1.430000000000001 + 0.5450000000000004 + 0.4870000000000003
saving weights of car 0 to template_2_weights.npy


In [2]:
'''
A template for training your cars using a Genetic Algorithm
'''

from template_1_car import MyCar
import numpy as np
from copy import deepcopy

def encode(weights, w_range, num_bits):
    '''
    Encode weights as an array of 1s and 0s
    '''
    encoded_weights = []
    int2binary = lambda x,bits: list(np.binary_repr(x).zfill(bits))
    min_w, max_w = w_range
    for w in weights:
        w = round((pow(2,num_bits)-1)*(w-min_w)/(max_w-min_w))
        encoded_weights.extend(int2binary(w, num_bits))
    return [int(i) for i in encoded_weights]

def decode(enc_weights,w_range,num_bits):
    '''
    Decode an array of 0s and 1s to the weights array
    '''
    n = len(enc_weights)
    bin2int = lambda x,bits: sum([np.power(2,bits-1-i)*x[i] for i in range(len(x))])
    min_w, max_w = w_range
    weights = []
    for i in range(0,n,num_bits):
        w = bin2int(enc_weights[i:i+num_bits], num_bits)
        w = min_w + (max_w-min_w)*w/(pow(2,num_bits)-1)
        weights.append(w)
    return np.array(weights)

def create_population(weight_size, hyper_params):
    '''
    create the GA population (essentially a list of cars)
    '''
    lo,hi = hyper_params['weight_range']
    return [MyCar(id=i, weights=np.random.uniform(lo,hi,weight_size)) for i in range(hyper_params['population_size'])]

def selection(population, track, hyper_params):
    '''
    Implement roulette wheel based selection for the entire population
    '''
    pop_size = hyper_params['population_size']
    num_elites = round(hyper_params['elite']*pop_size)
    sorted_pop = sorted(population, key = lambda p: p.run(track), reverse=True)
    elites = deepcopy(sorted_pop[:num_elites])
    pop_fitness = [p.run(track) + 1e-6 for p in population]
    new_pop = []

    new_pop.extend(elites) # put elites in the population

    while len(new_pop) < len(population):
        selected_member = np.random.choice(pop, p=np.array(pop_fitness)/sum(pop_fitness))
        new_pop.append(selected_member)

    return new_pop, elites

def crossover_util(w1, w2):
    '''
    Implement single point crossover for one pair of (encoded) weights
    '''
    crossover_point = np.random.choice(range(1,len(w1)-1))
    new_w = w1[:crossover_point] + w2[crossover_point:]
    return new_w

def crossover(population, hyper_params):
    '''
    Implement single point crossover for the entire population
    (parents are selected randomly)
    '''
    new_pop = []
    for i in range(hyper_params['population_size']):
        parent1 = np.random.choice(population)
        parent2 = np.random.choice(population)

        w1 = encode(parent1.weights, hyper_params['weight_range'], hyper_params['weight_bits'])
        w2 = encode(parent2.weights, hyper_params['weight_range'], hyper_params['weight_bits'])
        new_w = decode(crossover_util(w1,w2), hyper_params['weight_range'], hyper_params['weight_bits'])
        child = MyCar(id=i,weights=new_w)
        new_pop.append(child)

    return new_pop

def mutate(weights, mutation_prob):
    '''
    Mutate the given (enocoded) weights
    '''
    prob = hyper_params['mutation_prob']
    flip = lambda bit,p: 1-bit if np.random.random() < p else bit
    return [flip(w,prob) for w in weights]

def mutation(population, hyper_params):
    '''
    Implement uniform mutation for the entire population
    '''
    for i,member in enumerate(population):
        enc_weights = encode(member.weights, hyper_params['weight_range'], hyper_params['weight_bits']) # encode given weights
        mutated_weights = mutate(enc_weights,hyper_params['mutation_prob']) # mutate the encoded weights
        population[i].weights = decode(mutated_weights, hyper_params['weight_range'], hyper_params['weight_bits']) # decode the weights
    return population

def GA(population, track, hyper_params, print_every=10):
    pop = population
    f = [p.run(track) for p in pop]
    print(f'Iteration 0: [avg: {round(np.mean(f),3)} | best: {round(np.max(f),3)}]')

    for i in range(1,hyper_params['iterations']+1):
        pop,elites = selection(pop, track, hyper_params) # create parent population using selection (store the elites seperately)
        pop = crossover(pop, hyper_params) # Crossover
        pop = mutation(pop, hyper_params)

        num_elites = round(hyper_params['elite']*hyper_params['population_size'])
        pop = sorted(pop, key = lambda p: p.run(track))
        pop[:num_elites] = elites # propogate elite population without any changes

        if i%print_every == 0:
            f = [p.run(track) for p in pop]
            print(f'Iteration {i}: [avg: {round(np.mean(f),3)} | best: {round(np.max(f),3)}]')
            #print([p.run(track) for p in elites])
    return pop

### ----------------------------------------------------------------------- ###

# These can be freely changed
hyper_params = {
    'weight_range': (-3,3), # Range in which the wieghts can vary
    'weight_bits': 5, # Number of bits to use while encoding the weights
    'iterations': 15, # Number of iterations
    'population_size': 30, # Size of the population
    'elite': 0.15, # as a percentage of the population (between 0 and 1)
    'mutation_prob': 0.1 # mutation probability (between 0 and  1)
}

'''
Although weight_size can also be considered as a hyper parameters, it can't be
freely changed as changing this would require a corresponding change in your
car's move() function
'''
weight_size = 8

# create a population of cars
print('Creating the population ...')
pop = create_population(weight_size, hyper_params)

# train the Genetic algorithm on different tracks
track1 = 'tracks/sample_track_1.jpg'
print('Training on track 1 ...')
pop = GA(pop, track1, hyper_params, print_every=5)

track2 = 'tracks/sample_track_2.jpg'
print('Training on track 2 ...')
pop = GA(pop, track2, hyper_params, print_every=5)

# save the car that runs best on track 2
best_car = max(pop, key=lambda x: x.run(track2))
best_car.run(track2, save='best_car_ga_template_track_2.gif')
best_car.save(file='ga_best_weights')

# continue training on track 3
# car.load('ga_best_weights') # load the saved weights

print('Training on track 3 ...')
track3 = 'tracks/sample_track_3.jpg'
pop = GA(pop, track3, hyper_params, print_every=5)

# save the car that runs best on all tracks combined
best_car = max(pop, key=lambda x: x.run(track1) + x.run(track2) + x.run(track3))
f1 = best_car.run(track1, save='best_car_ga_template_track_1.gif')
f2 = best_car.run(track2, save='best_car_ga_template_track_2.gif')
f3 = best_car.run(track3, save='best_car_ga_template_track_3.gif')
print(f'Overall fitness: {f1+f2+f3} = {f1} + {f2} + {f3}')
best_car.save(file='ga_best_weights')


Creating the population ...
Training on track 1 ...
Iteration 0: [avg: 0.438 | best: 1.43]


/home/aadit/nnfl/Part-II/template_1_car.py:13: RuntimeWarning: overflow encountered in exp
  return (np.exp(z) - np.exp(-z))/(np.exp(z) + np.exp(-z))
/home/aadit/nnfl/Part-II/template_1_car.py:13: RuntimeWarning: invalid value encountered in double_scalars
  return (np.exp(z) - np.exp(-z))/(np.exp(z) + np.exp(-z))


Iteration 5: [avg: 0.791 | best: 1.5]
Iteration 10: [avg: 0.753 | best: 1.5]
Iteration 15: [avg: 0.561 | best: 1.5]
Training on track 2 ...
Iteration 0: [avg: 0.364 | best: 1.174]
Iteration 5: [avg: 0.642 | best: 1.215]
Iteration 10: [avg: 0.783 | best: 1.226]
Iteration 15: [avg: 0.558 | best: 1.246]


Visualizing fitness for car 21:  76%|███████▌  | 76/100 [00:03<00:01, 23.62it/s]


saving weights of car 21 to ga_best_weights.npy
Training on track 3 ...
Iteration 0: [avg: 0.274 | best: 0.968]
Iteration 5: [avg: 0.388 | best: 1.008]
Iteration 10: [avg: 0.404 | best: 0.995]
Iteration 15: [avg: 0.274 | best: 0.995]


Visualizing fitness for car 16: 100%|██████████| 100/100 [00:07<00:00, 14.11it/s]


Overall fitness: 3.088279009605123 = 1.4618563137490193 + 1.2636440116021639 + 0.3627786842539398
saving weights of car 16 to ga_best_weights.npy
